In [9]:
# Handwritten based OCR 


In [10]:
## Program to upload an OCR Image 
## Recognize Characters
## Save it in Text format

In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tkinter as tk
import cv2
import numpy as np
from tkinter import filedialog
from PIL import Image, ImageTk
from paddleocr import PaddleOCR

In [2]:
# Function for thinning the image
def thin_image(img):
    img_array = np.array(img)
    size = np.size(img_array)
    skel = np.zeros(img_array.shape, np.uint8)

    ret, img_array = cv2.threshold(img_array, 127, 255, 0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    done = False

    while not done:
        eroded = cv2.erode(img_array, element)
        temp = cv2.dilate(eroded, element)
        temp = cv2.subtract(img_array, temp)
        skel = cv2.bitwise_or(skel, temp)
        img_array = eroded.copy()

        zeros = size - cv2.countNonZero(cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY))
        if zeros == size:
            done = True

    return Image.fromarray(skel)

In [3]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # Thin the image
    #binary = thin(binary.copy())
    return Image.fromarray(binary)

In [4]:
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # You can set the language ('en', 'ch', etc.) according to your requirement

def load_image():
    global panel, img_label, text_box
    filename = filedialog.askopenfilename(initialdir="/", title="Select file",
                                          filetypes=(("jpeg files", "*.jpg"), ("png files", "*.png"), ("bmp files", "*.bmp")))
    if filename:
        img = Image.open(filename)
        img = img.resize((250, 250), Image.LANCZOS)  # Using LANCZOS resampling for resizing
        
        # Thin the image
        #img = cv2.thin(img.copy())
        #img = thin_image(img)
        img = ImageTk.PhotoImage(img)
        
        if panel is None:
            panel = tk.Label(image=img)
            panel.image = img
            panel.grid(row=0, column=0)
        else:
            panel.configure(image=img)
            panel.image = img

        result = ocr.ocr(filename, cls=True)
        
        # Display the image with the boxes
        panel.configure(image=img)
        
        text = '\n'.join([word[1][0] for line in result for word in line])
        text_box.delete("1.0", "end")
        text_box.insert(tk.END, text)

[2023/12/27 13:02:19] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\creat/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\creat/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None,

In [5]:
def convert_to_text_file():
    global text_box
    text = text_box.get("1.0", "end-1c")  # Retrieve the text without the trailing newline
    file_path = filedialog.asksaveasfilename(defaultextension=".txt")
    if file_path:
        with open(file_path, "w") as file:
            file.write(text)


In [6]:
def cancel():
    global panel, text_box
    if panel is not None:
        panel.grid_forget()
        panel = None
    text_box.delete("1.0", "end")


In [7]:

root = tk.Tk()
root.title("Image to Text Converter")

panel = None

upload_button = tk.Button(root, text="Upload", command=load_image)
upload_button.grid(row=1, column=0)

convert_button = tk.Button(root, text="Convert to Text", command=convert_to_text_file)
convert_button.grid(row=1, column=1)

cancel_button = tk.Button(root, text="Cancel", command=cancel)
cancel_button.grid(row=1, column=2)

text_box = tk.Text(root, height=10, width=40)
text_box.grid(row=0, column=1)

root.mainloop()


[2023/12/27 13:02:45] ppocr DEBUG: dt_boxes num : 45, elapsed : 0.5782983303070068
[2023/12/27 13:02:45] ppocr DEBUG: cls num  : 45, elapsed : 0.35642409324645996
[2023/12/27 13:02:52] ppocr DEBUG: rec_res num  : 45, elapsed : 6.433444261550903
